In [21]:
# Impoting Library

import pandas as pd
import numpy as np
import re
import nltk
import string
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Roy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Roy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
## Reading in Data
fake = pd.read_csv("Dataset/Fake.csv")
true = pd.read_csv("Dataset/True.csv")
stop = stopwords.words('english')

In [23]:
# Encoding the data
fake['Label'] = 1
true['Label'] = 0

In [24]:
# Joining both true and false into one DF
data = pd.concat([true,fake],axis=0,ignore_index=True)

In [25]:
# Moving all important features into a single column 

data['text'] = data['title'] + data['text']
data = data.drop(['title'],axis=1)

In [31]:
### Cleaning Tokenizing and Lemmatizing
### Cleaning, Tokenizing and Lemmatizing
stop = stopwords.words('english')
def clean_up(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
data['text'] = data['text'].apply(lambda x:clean_up(x))
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [32]:
def lemmatize_words(text):
    wnl = nltk.stem.WordNetLemmatizer()
    lematized = ' '.join([wnl.lemmatize(word) for word in text.split()])    
    return lematized
data['text'] = data['text'].apply(lemmatize_words)

In [33]:
## Creating Training and Testing Sets
y = data['Label']
X_train, X_test, y_train, y_test = train_test_split(data['text'], y,test_size=0.40,random_state=53)

In [34]:
## Creating a Bag of Words and Converting Feature into Numbers to feed into the model.
count_vectorizer = CountVectorizer(stop_words='english')
count_train = count_vectorizer.fit_transform(X_train.values)
count_test = count_vectorizer.transform(X_test.values)
print(count_train.shape)

(26938, 162434)


In [35]:
# Time for some fun stuff, training Naive Bayes
from sklearn.metrics import classification_report
nb_classifier1 = MultinomialNB()
nb_classifier1.fit(count_train, y_train)
pred1 = nb_classifier1.predict(count_test)
print(classification_report(y_test, pred1, target_names = ['Fake','True']))

              precision    recall  f1-score   support

        Fake       0.95      0.96      0.95      8659
        True       0.96      0.95      0.96      9301

    accuracy                           0.95     17960
   macro avg       0.95      0.95      0.95     17960
weighted avg       0.95      0.95      0.95     17960

